In [1]:
import pandas as pd
import numpy as np
import sklearn
import scipy

import anndata as ad
import scanpy as sc

from dask import delayed
from dask.distributed import Client, LocalCluster

import os, binascii

In [ ]:
#! aws s3 cp s3://saturn-kaggle-datasets/open-problems-single-cell-perturbations-optional/lincs_id_compound_mapping.parquet --no-sign-request .

## Loading expression data

Here we load expression data (long format) and converting it into an AnnData object (wide sparse format).

You'll need to increase your instance RAM to at least 64 GB.

In [3]:
data_dir = '/home/jovyan/kaggle/input/open-problems-single-cell-perturbations'

adata_train_df = pd.read_parquet(os.path.join(data_dir, 'adata_train.parquet'))
adata_obs_meta_df = pd.read_csv(os.path.join(data_dir, 'adata_obs_meta.csv'))
adata_excluded_ids_df = pd.read_csv(os.path.join(data_dir, 'adata_excluded_ids.csv'))

In [5]:
reindexed_adata_train_df = adata_train_df.reset_index().set_index(['obs_id', 'gene'])
adata_excluded_ids_index = adata_excluded_ids_df.set_index(['obs_id', 'gene']).sort_index().index

selection = ~reindexed_adata_train_df.index.isin(adata_excluded_ids_index)
adata_train_df = reindexed_adata_train_df[selection]

adata_train_df = adata_train_df.reset_index().set_index('index')

In [7]:
adata_train_df['obs_id'] = adata_train_df['obs_id'].astype('category')
adata_train_df['gene'] = adata_train_df['gene'].astype('category')

obs_ids = adata_train_df['obs_id'].unique()
obs_id_map = dict(zip(obs_ids, range(len(obs_ids))))

genes = adata_train_df['gene'].unique()
gene_map = dict(zip(genes, range(len(genes))))

adata_train_df['obs_index'] = adata_train_df['obs_id'].map(obs_id_map)
adata_train_df['gene_index'] = adata_train_df['gene'].map(gene_map)

normalized_counts_values = adata_train_df['normalized_count'].to_numpy()
counts_values = adata_train_df['count'].to_numpy()

row_indices = adata_train_df['obs_index'].to_numpy()
col_indices = adata_train_df['gene_index'].to_numpy()

counts = scipy.sparse.csr_matrix((counts_values, (row_indices, col_indices)))

obs_df = pd.Series(obs_ids, name='obs_id').to_frame()
var_df = pd.Series(genes, name='gene').to_frame()

obs_df = obs_df.set_index('obs_id')
var_df = var_df.set_index('gene')

obs_df.index = obs_df.index.astype('str')
var_df.index = var_df.index.astype('str')

kaggle_adata = ad.AnnData(
    X=counts,
    obs=obs_df,
    var=var_df,
    dtype=np.uint32,
)

index_ordering_before_join = kaggle_adata.obs.index
kaggle_adata.obs = kaggle_adata.obs.join(adata_obs_meta_df.set_index('obs_id'))
index_ordering_after_join = kaggle_adata.obs.index
assert (index_ordering_before_join == index_ordering_after_join).all()

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


In [8]:
kaggle_adata

AnnData object with n_obs × n_vars = 240090 × 18211
    obs: 'library_id', 'plate_name', 'well', 'row', 'col', 'cell_id', 'donor_id', 'cell_type', 'sm_lincs_id', 'sm_name', 'SMILES', 'dose_uM', 'timepoint_hr', 'control'

## Pseudobulking counts by cell type

In [9]:
from scipy import sparse

def sum_by(adata: ad.AnnData, col: str) -> ad.AnnData:
    """
    Adapted from this forum post: 
    https://discourse.scverse.org/t/group-sum-rows-based-on-jobs-feature/371/4
    """
    
    assert pd.api.types.is_categorical_dtype(adata.obs[col])

    # sum `.X` entries for each unique value in `col`
    cat = adata.obs[col].values
    indicator = sparse.coo_matrix(
        (
            np.broadcast_to(True, adata.n_obs),
            (cat.codes, np.arange(adata.n_obs))
        ),
        shape=(len(cat.categories), adata.n_obs),
    )
    sum_adata = ad.AnnData(
        indicator @ adata.X,
        var=adata.var,
        obs=pd.DataFrame(index=cat.categories),
    )
    
    # copy over `.obs` values that have a one-to-one-mapping with `.obs[col]`
    obs_cols = adata.obs.columns
    obs_cols = list(set(adata.obs.columns) - set([col]))
    
    one_to_one_mapped_obs_cols = []
    nunique_in_col = adata.obs[col].nunique()
    for other_col in obs_cols:
        if len(adata.obs[[col, other_col]].drop_duplicates()) == nunique_in_col:
            one_to_one_mapped_obs_cols.append(other_col)

    joining_df = adata.obs[[col] + one_to_one_mapped_obs_cols].drop_duplicates().set_index(col)
    assert (sum_adata.obs.index == sum_adata.obs.join(joining_df).index).all()
    sum_adata.obs = sum_adata.obs.join(joining_df)
    sum_adata.obs.index.name = col
    sum_adata.obs = sum_adata.obs.reset_index()
    sum_adata.obs.index = sum_adata.obs.index.astype('str')

    return sum_adata

In [10]:
kaggle_adata.obs['plate_well_cell_type'] = kaggle_adata.obs['plate_name'].astype('str') \
    + '_' + kaggle_adata.obs['well'].astype('str') \
    + '_' + kaggle_adata.obs['cell_type'].astype('str')
kaggle_adata.obs['plate_well_cell_type'] = kaggle_adata.obs['plate_well_cell_type'].astype('category')

bulk_adata = sum_by(kaggle_adata, 'plate_well_cell_type')
bulk_adata.obs = bulk_adata.obs.drop(columns=['plate_well_cell_type'])
bulk_adata.X = np.array(bulk_adata.X.todense())
bulk_adata = bulk_adata.copy()

In [11]:
plate_name_update = {
    'plate_0': 'plate_2',
     'plate_1': 'plate_3',
     'plate_2': 'plate_1',
     'plate_3': 'plate_4',
     'plate_4': 'plate_0',
     'plate_5': 'plate_5',
}

bulk_adata.obs['plate_name'] = bulk_adata \
    .obs['plate_name'] \
    .map(plate_name_update) \
    .astype('str')

## Running Limma

In [12]:
de_pert_cols = [
    'sm_name',
    'sm_lincs_id',
    'SMILES',
    'dose_uM',
    'timepoint_hr',
    'cell_type',
]

control_compound = 'Dimethyl Sulfoxide'

In [13]:
import limma_utils
import imp
imp.reload(limma_utils)

/tmp/ipykernel_719/3906383884.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


<module 'limma_utils' from '/home/jovyan/kaggle/neurips-2023-scripts/limma_utils.py'>

In [14]:
!mkdir -p output

In [15]:
def _run_limma_for_cell_type(bulk_adata):
    import limma_utils
    bulk_adata = bulk_adata.copy()
    
    compound_name_col = de_pert_cols[0]
    
    # limma doesn't like dashes etc. in the compound names
    rpert_mapping = bulk_adata.obs[compound_name_col].drop_duplicates() \
        .reset_index(drop=True).reset_index() \
        .set_index(compound_name_col)['index'].to_dict()
    
    bulk_adata.obs['Rpert'] = bulk_adata.obs.apply(
        lambda row: rpert_mapping[row[compound_name_col]], 
        axis='columns',
    ).astype('str')

    compound_name_to_Rpert = bulk_adata.obs.set_index(compound_name_col)['Rpert'].to_dict()
    ref_pert = compound_name_to_Rpert[control_compound]
            
    random_string = binascii.b2a_hex(os.urandom(15)).decode()
    
    
    limma_utils.limma_fit(
        bulk_adata, 
        design='~0+Rpert+donor_id+plate_name+row',
        output_path=f'output/{random_string}_limma.rds',
        plot_output_path=f'output/{random_string}_voom',
        exec_path='limma_fit.r',
    )

    pert_de_dfs = []
    


    for pert in bulk_adata.obs['Rpert'].unique():
        if pert == ref_pert:
            continue

        pert_de_df = limma_utils.limma_contrast(
            fit_path=f'output/{random_string}_limma.rds',
            contrast='Rpert'+pert+'-Rpert'+ref_pert,
            exec_path='limma_contrast.r',
        )

        pert_de_df['Rpert'] = pert

        pert_obs = bulk_adata.obs[bulk_adata.obs['Rpert'].eq(pert)]
        for col in de_pert_cols:
            pert_de_df[col] = pert_obs[col].unique()[0]
        pert_de_dfs.append(pert_de_df)

    de_df = pd.concat(pert_de_dfs, axis=0)

    try:
        os.remove(f'output/{random_string}_limma.rds')
        os.remove(f'output/{random_string}_voom')
    except FileNotFoundError:
        pass
    
    return de_df

run_limma_for_cell_type = delayed(_run_limma_for_cell_type)

In [16]:
%%capture

cluster = LocalCluster(
    n_workers=6,
    processes=True,
    threads_per_worker=1,
    memory_limit='20GB',
)

c = Client(cluster)

2023-11-22 14:36:21,307 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/kaggle/neurips-2023-scripts/dask-worker-space/worker-8mb8mmvg', purging
2023-11-22 14:36:21,309 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/kaggle/neurips-2023-scripts/dask-worker-space/worker-qydrcq_y', purging
2023-11-22 14:36:21,309 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/kaggle/neurips-2023-scripts/dask-worker-space/worker-tdn1jf0q', purging
2023-11-22 14:36:21,310 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/kaggle/neurips-2023-scripts/dask-worker-space/worker-muo0_oyi', purging
2023-11-22 14:36:21,310 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovyan/kaggle/neurips-2023-scripts/dask-worker-space/worker-asoh903h', purging
2023-11-22 14:36:21,310 - distributed.diskutils - INFO - Found stale lock file and directory '/home/jovya

We compute DE for each cell type independently. Using Dask, we execute the DE expression function for each cell type in parallel.

In [17]:
%%capture

cell_types = bulk_adata.obs['cell_type'].unique()
de_dfs = []

for cell_type in cell_types:
    cell_type_selection = bulk_adata.obs['cell_type'].eq(cell_type)
    cell_type_bulk_adata = bulk_adata[cell_type_selection].copy()
    
    de_df = run_limma_for_cell_type(cell_type_bulk_adata)
    
    de_dfs.append(de_df)

de_dfs = c.compute(de_dfs, sync=True)
de_df = pd.concat(de_dfs)

## Converting DataFrame to Anndata

In [18]:
def convert_de_df_to_anndata(de_df, pert_cols, de_sig_cutoff):
    de_df = de_df.copy()
    zero_pval_selection = de_df['P.Value'].eq(0)
    de_df.loc[zero_pval_selection, 'P.Value'] = np.finfo(np.float64).eps

    de_df['sign_log10_pval'] = np.sign(de_df['logFC']) * -np.log10(de_df['P.Value'])
    de_df['is_de'] = de_df['P.Value'].lt(de_sig_cutoff)
    de_df['is_de_adj'] = de_df['adj.P.Val'].lt(de_sig_cutoff)

    de_feature_dfs = {}
    for feature in ['is_de', 'is_de_adj', 'sign_log10_pval', 'logFC', 'P.Value', 'adj.P.Val']:
        df = de_df.reset_index().pivot_table(
            index=['gene'], 
            columns=pert_cols,
            values=[feature],
            dropna=True,
        )
        de_feature_dfs[feature] = df

    de_adata = ad.AnnData(de_feature_dfs['sign_log10_pval'].T, dtype=np.float64)
    de_adata.obs = de_adata.obs.reset_index()
    de_adata.obs = de_adata.obs.drop(columns=['level_0'])
    de_adata.obs.index = de_adata.obs.index.astype('string')

    de_adata.layers['is_de'] = de_feature_dfs['is_de'].to_numpy().T
    de_adata.layers['is_de_adj'] = de_feature_dfs['is_de_adj'].to_numpy().T
    de_adata.layers['logFC'] = de_feature_dfs['logFC'].to_numpy().T
    de_adata.layers['P.Value'] = de_feature_dfs['P.Value'].to_numpy().T
    de_adata.layers['adj.P.Val'] = de_feature_dfs['adj.P.Val'].to_numpy().T
    
    return de_adata

In [19]:
de_adata = convert_de_df_to_anndata(de_df, de_pert_cols, 0.05)

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(


## Validation

Let's make sure that the DE object we've computed has the exact same values as the DE object obtained from `de_train.parquet`

In [20]:
kaggle_train_de_df = pd.read_parquet(os.path.join(data_dir, 'de_train.parquet'))
kaggle_train_de_df = kaggle_train_de_df.set_index(list(kaggle_train_de_df.columns[:5]))

kaggle_train_de_adata = ad.AnnData(kaggle_train_de_df)
kaggle_train_de_adata.obs = kaggle_train_de_adata.obs.reset_index()
kaggle_train_de_adata.obs.index = kaggle_train_de_adata.obs.index.astype('str')

In [21]:
sorting_index = kaggle_train_de_adata.obs.sort_values(['sm_name', 'cell_type']).index
kaggle_train_de_adata = kaggle_train_de_adata[sorting_index].copy()

In [22]:
de_adata.obs.index = de_adata.obs.index.astype('str')

sorting_index = de_adata.obs.sort_values(['sm_name', 'cell_type']).index
de_adata = de_adata[sorting_index].copy()

In [23]:
kaggle_train_de_adata.obs

,cell_type,sm_name,sm_lincs_id,SMILES,control
590,NK cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,LSM-47134,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,False
591,T cells CD4+,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,LSM-47134,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,False
592,T cells CD8+,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,LSM-47134,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,False
593,T regulatory cells,5-(9-Isopropyl-8-methyl-2-morpholino-9H-purin-...,LSM-47134,Cc1nc2c(-c3cnc(N)nc3)nc(N3CCOCC3)nc2n1C(C)C,False
56,NK cells,ABT-199 (GDC-0199),LSM-45468,CC1(C)CCC(CN2CCN(c3ccc(C(=O)NS(=O)(=O)c4ccc(NC...,False
...,...,...,...,...,...
67,T regulatory cells,Vorinostat,LSM-3828,O=C(CCCCCCC(=O)Nc1ccccc1)NO,False
523,NK cells,YK 4-279,LSM-4933,COc1ccc(C(=O)CC2(O)C(=O)Nc3c(Cl)ccc(Cl)c32)cc1,False
524,T cells CD4+,YK 4-279,LSM-4933,COc1ccc(C(=O)CC2(O)C(=O)Nc3c(Cl)ccc(Cl)c32)cc1,False
525,T cells CD8+,YK 4-279,LSM-4933,COc1ccc(C(=O)CC2(O)C(=O)Nc3c(Cl)ccc(Cl)c32)cc1,False


In [24]:
np.allclose(kaggle_train_de_adata.X, de_adata.X)

True